In [2]:
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import requests
import os
import urllib.request
from urllib.request import urlretrieve
import pandas as pd
import re
import pickle

#### Build initial list of URLs

In [241]:
# url_list_s1 = []
url_list_s1

['https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=1&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=81&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=161&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=241&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=321&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=401&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=481&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=561&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=641&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=721&count=80&first=2018&last=2019',
 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=801&count=80&first=2018&las

In [229]:
# for S-1
# https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&first=2018&last=2019

count = 1
while count < 1290:
    url_test = 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=' + str(count) + '&count=80&first=2018&last=2019'
    response = requests.get(url_test)
    if response.status_code == 200:
        url_list_s1.append(url_test)
        count += 80


In [231]:
with open('url_list_s1.pkl', 'wb') as f:
    pickle.dump(url_list_s1, f)

In [250]:
url_list_s1a = []

In [251]:
# for S-1/A
# https://www.sec.gov/cgi-bin/srch-edgar?text=S-1%2FA&first=2018&last=2019

count = 1
while count < 2350:
    url_test = 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1%2FA&start=' + str(count) + '&count=80&first=2018&last=2019'
    response = requests.get(url_test)
    if response.status_code == 200:
        url_list.append(url_test)
        count += 80



In [256]:
with open('url_list_s1a.pkl', 'wb') as f:
    pickle.dump(url_list, f)

#### Create Dataframes

In [257]:
df_s1a = pd.DataFrame(columns = ['No.','Company','Format','Form Type','Filing Date','Size'])

for url in url_list:
    try:
        response = requests.get(url)
        page = response.text
        df_2 = pd.read_html(page)[4]

        s1_urls = []

        soup= BeautifulSoup(page,'lxml')
        main_table = soup.find_all('table')[4]

        for j in main_table.find_all('td'):
            for k in j.find_all('a'):   
                if k.text == "[text]":
                    value = "https://www.sec.gov" + k['href']
                    s1_urls.append(value)
                    url_series = pd.Series(s1_urls)
        temp_df = pd.concat([df_2,url_series],axis=1)
        df_s1a = df_s1a.append(temp_df, ignore_index = True)

    except ValueError:
        pass

In [258]:
# df_s1.shape

df_s1a.to_pickle("./dataframe_2.pkl")

#### Concatenate dataframes

In [269]:
df = df_s1.append(df_s1a, ignore_index=True)

In [272]:
df.to_pickle("./dataframe_full.pkl")

In [28]:
df2.columns = ['company', 'filing_date', 'form_type', 'format', 'no.', 'size', 'url']

#### Drop NaNs, convert date

In [278]:
df.shape

(3366, 7)

In [24]:
df.shape

(3366, 7)

In [3]:
df = pd.read_pickle('./dataframe_full.pkl')

In [25]:
df2 = df.dropna()

In [29]:
df2.isna().sum()

company        0
filing_date    0
form_type      0
format         0
no.            0
size           0
url            0
dtype: int64

In [60]:
df2.head()

,company,filing_date,form_type,format,no.,size,url
0,10SION Holdings Inc.,2018-10-29,S-1,[text] [html],320,979199,https://www.sec.gov/Archives/edgar/data/175668...
1,12 Retech Corp,2018-07-02,S-1,[text] [html],320,6397781,https://www.sec.gov/Archives/edgar/data/162761...
2,"1347 Property Insurance Holdings, Inc.",2018-01-08,S-1,[text] [html],320,2469600,https://www.sec.gov/Archives/edgar/data/159189...
3,"1895 Bancorp of Wisconsin, Inc.",2018-09-07,S-1,[text] [html],320,7184379,https://www.sec.gov/Archives/edgar/data/175169...
4,8i Enterprises Acquisition Corp.,2019-02-22,S-1,[text] [html],320,1360947,https://www.sec.gov/Archives/edgar/data/175364...


In [32]:
df2_group = df2.groupby(['company'])

In [35]:
df2['filing_date'] = pd.to_datetime(df2['filing_date'])


/Users/kathmbell/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [93]:
recent_dates_by_company = df2.groupby(['company'])['filing_date'].max().reset_index()
# recent_dates_by_company.info()
recent_dates_by_company.shape

(1049, 2)

In [163]:
# df_merge = pd.merge_asof(df2, recent_dates_by_company,
#               on='filing_date',
#               by='company')

new_df = pd.merge(recent_dates_by_company, df2,
                  how = 'left'
                  ,on=['company','filing_date']
                 )
df3 = new_df.drop_duplicates()


In [169]:
duplicateRowsDF = df3[df3.duplicated(['company'])]
duplicateRowsDF

,company,filing_date,form_type,format,no.,size,url
49,ALLIANZ LIFE INSURANCE CO OF NORTH AMERICA,2019-04-16,S-1,[text] [html],320,11046331,https://www.sec.gov/Archives/edgar/data/72499/...
50,ALLIANZ LIFE INSURANCE CO OF NORTH AMERICA,2019-04-16,S-1,[text] [html],320,10794978,https://www.sec.gov/Archives/edgar/data/72499/...
177,"Athenex, Inc.",2018-01-22,S-1/A,[text] [html],377,380027,https://www.sec.gov/Archives/edgar/data/130069...
183,"Avantor, Inc.",2019-05-16,S-1/A,[text] [html],377,104066,https://www.sec.gov/Archives/edgar/data/172248...
382,"DigitalTown, Inc.",2018-09-12,S-1/A,[text] [html],377,7709799,https://www.sec.gov/Archives/edgar/data/106559...
413,Elite Performance Holding Corp,2019-04-10,S-1/A,[text] [html],377,1538460,https://www.sec.gov/Archives/edgar/data/175368...
420,"Energy Conversion Services, Inc.",2018-03-15,S-1/A,[text] [html],377,532205,https://www.sec.gov/Archives/edgar/data/171932...
541,"Guardion Health Sciences, Inc.",2019-08-12,S-1/A,[text] [html],377,308492,https://www.sec.gov/Archives/edgar/data/164237...
641,"Jerrick Media Holdings, Inc.",2018-11-30,S-1/A,[text] [html],377,12547734,https://www.sec.gov/Archives/edgar/data/135767...
665,LINCOLN BENEFIT LIFE CO,2018-04-02,S-1,[text] [html],320,10813188,https://www.sec.gov/Archives/edgar/data/910739...


In [171]:
df_test = df3[:10]
df_test.shape

(10, 7)

#### Get data from site, append df

In [194]:
data_list = []

for index, row in df_test.iterrows():
    try:
        page = urllib.request.urlopen(row['url'])
        contents = str(page.read())
        soup = BeautifulSoup(contents)
        soup = soup.text
        result = re.search(r'PROS([\s\S]*)RISK', soup).group(1)
        result = result.replace('\\n',' ')
        result = result.replace('\xa0',' ')
        data_list.append(result)
    except AttributeError:
        data_list.append('')
    

In [196]:
df_test['text'] = data_list

/Users/kathmbell/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [198]:
df_test

,company,filing_date,form_type,format,no.,size,url,text
0,10SION Holdings Inc.,2018-10-29,S-1,[text] [html],320,979199,https://www.sec.gov/Archives/edgar/data/175668...,PECTUS SUBJECT TO COMPLETION - DATED OCTOBER _...
2,12 Retech Corp,2018-07-02,S-1,[text] [html],320,6397781,https://www.sec.gov/Archives/edgar/data/162761...,PECTUS SUMMARY 1
4,"1347 Property Insurance Holdings, Inc.",2018-01-08,S-1,[text] [html],320,2469600,https://www.sec.gov/Archives/edgar/data/159189...,PECTUS SUBJECT TO COMPLETION ...
6,"1895 Bancorp of Wisconsin, Inc.",2018-09-07,S-1,[text] [html],320,7184379,https://www.sec.gov/Archives/edgar/data/175169...,
8,8i Enterprises Acquisition Corp.,2019-02-22,S-1,[text] [html],320,1360947,https://www.sec.gov/Archives/edgar/data/175364...,"PECTUS SUBJECT TO COMPLETION, DATED ..."
10,"ABCO Energy, Inc.",2019-04-26,S-1,[text] [html],320,5201748,https://www.sec.gov/Archives/edgar/data/130093...,PECTUS IS NOT COMPLETE AND MAY BE CHANGED. WE ...
12,"ACHIEVE LIFE SCIENCES, INC.",2019-07-25,S-1,[text] [html],320,748065,https://www.sec.gov/Archives/edgar/data/949858...,PECTUS SUMMARY 2 RISK FACTOR...
14,"ACORN ENERGY, INC.",2019-05-03,S-1,[text] [html],320,520920,https://www.sec.gov/Archives/edgar/data/880984...,"PECTUS SUBJECT TO COMPLETION, DATED MAY 3, 201..."
16,ACQUIRED SALES CORP,2019-08-02,S-1,[text] [html],320,1194552,https://www.sec.gov/Archives/edgar/data/139113...,"PECTUS SUBJECT TO COMPLETION DATED July 26, 20..."
18,"ACRO BIOMEDICAL CO., LTD.",2018-02-15,S-1,[text] [html],320,1838296,https://www.sec.gov/Archives/edgar/data/162299...,"PECTUS, SUBJECT TO COMPLETION, DATED FEBRUARY ..."


In [ ]:
page = urllib.request.urlopen('https://www.sec.gov/Archives/edgar/data/1756680/000157570518000214/0001575705-18-000214.txt')
contents = str(page.read())
soup = BeautifulSoup(contents)
soup = soup.text
result = re.search(r'PROS([\s\S]*)RISK', soup).group(1)
print(result)

In [ ]:
path = "/Users/kathmbell/Dropbox/project_04/test_folder"
page = urllib.request.urlopen('https://www.sec.gov/Archives/edgar/data/1727815/000147793218002883/0001477932-18-002883.txt')
name = "company_name.txt"
completeName = os.path.join(path, name) 
f = open(completeName,'w')
content = str(page.read())
# contents = Path(name).read_text()
soup = BeautifulSoup(contents)
soup = soup.text
result = re.search(r'PROS([\s\S]*)RISK', soup).group(1)
f.write(result)
f.close()

In [ ]:
pd.merge(df_a, df_b, on='subject_id', how='left', suffixes=('_left', '_right'))

In [ ]:
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id')
df1.merge(df2.assign(val3 = 1), on=['index', 'val2'], how='left')


In [101]:
recent_dates_by_company.nunique()

company        1049
filing_date     362
dtype: int64

In [ ]:
response = requests.get('https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=1&count=80&first=2018&last=2019')
page = response.text
df_2 = pd.read_html(page)[4]


In [239]:
s1_urls = []
    
soup= BeautifulSoup(page,'lxml')
main_table = soup.find_all('table')[4]
for j in main_table.find_all('td'):
    for k in j.find_all('a'):   
        if k.text == "[text]":
            value = "https://www.sec.gov" + k['href']
            s1_urls.append(value)
            url_series = pd.Series(s1_urls)
temp_df = pd.concat([df_2,url_series],axis=1)

In [244]:
df_s1.shape

(972, 7)

In [205]:
df = pd.DataFrame(columns = ['No.','Company','Format','Form Type','Filing Date','Size'])

for url in url_list:
    response = requests.get(url)
    page = response.text
    df_2 = pd.read_html(page)[4]
    
    s1_urls = []
    soup= BeautifulSoup(page,'lxml')
    main_table = soup.find_all('table')[4]

    for j in main_table.find_all('td'):
        for k in j.find_all('a'):   
            if k.text == "[text]":
                value = "https://www.sec.gov" + k['href']
                s1_urls.append(value)
#     url_series = pd.Series(s1_urls)
#     temp_df = pd.concat([df_2,url_series],axis=1)
#     df.append(temp_df, ignore_index = True)
    
#     soup= BeautifulSoup(page,'lxml')
#     s1_urls = []
#     for table in soup.find_all('table'):
#         for j in tag.find_all('td'):
#             for k in j.find_all('a'):   
#                 if k.text == "[text]":
#                     value = "https://www.sec.gov" + k['href']
#                     print(value)
# #                     s1_urls.append(value)
#     url_series = pd.Series(s1_urls)
# #     temp_df = pd.concat([df_2,url_series],axis=1)
# #     df.append(temp_df, ignore_index = True)
    
                    
    

# df[‘Filing Date’] = pd.to_datetime(df[‘Filing Date’])
# recent_dates_by_company = df.groupby([‘Company’])[‘Filing Date’].max()
# recent_dates_by_company

,No.,Company,Format,Form Type,Filing Date,Size,S1-URL


In [191]:
response = requests.get('https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=1&count=80&first=2018&last=2019')
page = response.text
df_2 = pd.read_html(page)[4]

In [217]:



# for table in soup.find_all('table'):
#     print(table[4])
#     for j in tag.find_all('td'):
#         for k in j.find_all('a'):   
#             if k.text == "[text]":
#                 value = "https://www.sec.gov" + k['href']
#                 s1_urls.append(value)
# #     temp_df = pd.concat([df_2,url_series],axis=1)
# #     df.append(temp_df, ignore_index = True)

ValueError: No tables found

In [198]:
url_series.head()

Series([], dtype: float64)

In [ ]:
date_list = []
td_list = []
hyperlink_text = []
company_set = set()

hyperlink_dict = {}

date = 'string'

for url in url_list:
    response = requests.get(url)
    page = response.text
    soup= BeautifulSoup(page,'lxml')
    for tag in soup.find_all('tr'):
        for j in tag.find_all('td'):
            if ('/2019' in j.text) or ('/2018' in j.text):
                date = j.text
#                 print(date)
# #                 date = date.replace("/", "")
# #                 date = date[-4:] + date[0:4]
# #                 date_list.append(date)   
            for k in j.find_all('a'):   
                if ("Search" not in k.text) and ("Home" not in k.text) and ("Previous" not in k.text) and ("text" not in k.text) and ("html" not in k.text) and (k.text != "\n"):
                    company_name =  k.text # this gets company name and date submitted as key
                    company_name = company_name.replace(" ", "_")
                    company_name = company_name.replace(".", "")
                    company_name = company_name.replace("/", "--")
                    company_name = company_name.replace(",", "")
                    print(company_name, date)
#                     temp_tuple = (company_name, date)
#                     print(temp_tuple)
#                     td_list.append(temp_tuple)
#                 if k.text == "[text]":
#                     value = "https://www.sec.gov" + k['href']
#                     hyperlink_text.append(value)

# hyperlink_dict = dict(zip(td_list, hyperlink_text))




In [97]:
len(hyperlink_dict)

65

In [ ]:
hyperlink_dict

In [ ]:
path = "/Users/kathmbell/Dropbox/project_04/s1_partial"

# change this to append a dictionary

for key, url in hyperlink_dict.items():
    page = urllib.request.urlopen(url)
    name = key + ".txt"
    completeName = os.path.join(path, name) 
    f = open(completeName,'w')
    content = str(page.read())
# contents = Path(name).read_text()
    soup = BeautifulSoup(contents)
    soup = soup.text
    result = re.search(r'PROS([\s\S]*)RISK', soup).group(1)
    f.write(result)
    f.close()

In [57]:
# path = "/Users/kathmbell/Dropbox/project_04"

# for key, url in hyperlink_dict.items():
#     page = urllib.request.urlopen(url)
#     name = key + ".txt"
#     completeName = os.path.join(path, name) 
#     f = open(completeName,'w')
#     content = str(page.read())
#     contents = Path(content).read_text()
#     soup = BeautifulSoup(contents)
#     result = re.search(r'PROS([\s\S]*)RISK', soup).group(0)
#     f.write(result)
#     f.close()

# path = "/Users/kathmbell/Dropbox/project_04/test_folder"
# page = urllib.request.urlopen('https://www.sec.gov/Archives/edgar/data/1727815/000147793218002883/0001477932-18-002883.txt')
# name = "company_name.txt"
# completeName = os.path.join(path, name) 
# f = open(completeName,'w')
# content = str(page.read())
# # contents = Path(name).read_text()
# soup = BeautifulSoup(contents)
# soup = soup.text
# result = re.search(r'PROS([\s\S]*)RISK', soup).group(1)
# f.write(result)
# f.close()

In [8]:
f = open('test_file.txt',"r")
# print(f.readlines())

In [51]:
from pathlib import Path
from bs4 import BeautifulSoup

contents = Path('test_file.txt').read_text()
soup=BeautifulSoup(contents)
# soup.findAll('Summary')
soup_text = soup.text

In [7]:
from pathlib import Path
from bs4 import BeautifulSoup

contents = Path('test_file.txt').read_text()
soup=BeautifulSoup(contents)
soup = soup.text

In [50]:
import re


result = re.search(r'PROS([\s\S]*)RISK', soup).group(0)
# print(result)

# result = re.search(r'PROSPECTUS(.*)RISK', soup)


42327

In [65]:
path = "/Users/kathmbell/Dropbox/project_04/test_folder"
page = urllib.request.urlopen('https://www.sec.gov/Archives/edgar/data/1727815/000147793218002883/0001477932-18-002883.txt')
name = "company_name.txt"
completeName = os.path.join(path, name) 
f = open(completeName,'w')
content = str(page.read())
# contents = Path(name).read_text()
soup = BeautifulSoup(contents)
soup = soup.text
result = re.search(r'PROS([\s\S]*)RISK', soup).group(1)
f.write(result)
f.close()

In [124]:
df = pd.read_html('https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=1&count=80&first=2018&last=2019')

In [138]:
response = requests.get('https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=1&count=80&first=2018&last=2019')
page = response.text
soup= BeautifulSoup(page,'lxml')

#         for j in tag.find_all('td'):
#             if ('/2019' in j.text) or ('/2018' in j.text):

IndexError: list index out of range

In [152]:
main_table = soup.find_all('table')[4]


In [154]:
dfs = pd.read_html(str(main_table))[0]


In [155]:
dfs

,No.,Company,Format,Form Type,Filing Date,Size
0,320,10SION Holdings Inc.,[text] [html],S-1,10/29/2018,979199
1,320,12 Retech Corp,[text] [html],S-1,07/02/2018,6397781
2,320,"1347 Property Insurance Holdings, Inc.",[text] [html],S-1,01/08/2018,2469600
3,320,"1895 Bancorp of Wisconsin, Inc.",[text] [html],S-1,09/07/2018,7184379
4,320,8i Enterprises Acquisition Corp.,[text] [html],S-1,02/22/2019,1360947
5,320,"ABCO Energy, Inc.",[text] [html],S-1,04/26/2019,5201748
6,320,Abpro Corp,[text] [html],S-1,04/11/2018,6531108
7,320,"Acacia Diversified Holdings, Inc.",[text] [html],S-1,02/14/2019,8170560
8,320,"Acacia Diversified Holdings, Inc.",[text] [html],S-1,04/18/2018,5583779
9,320,Acamar Partners Acquisition Corp.,[text] [html],S-1,01/08/2019,1805513


In [ ]:
# page = urllib.request.urlopen(url)
# name = key + ".txt"
# completeName = os.path.join(path, name) 
# f = open(completeName,'w')
# content = str(page.read())
# contents = Path(name).read_text()
soup = BeautifulSoup(contents)
soup = soup.text
result = re.search(r'PROS([\s\S]*)RISK', soup).group(1)

In [8]:
url = 'https://www.sec.gov/Archives/edgar/data/1727815/000147793218002883/0001477932-18-002883.txt'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')
print(soup.prettify())

<html>
 <body>
  <sec-document>
   0001477932-18-002883.txt : 20180605
   <sec-header>
    0001477932-18-002883.hdr.sgml : 20180605
    <acceptance-datetime>
     20180604180740
ACCESSION NUMBER:		0001477932-18-002883
CONFORMED SUBMISSION TYPE:	S-1/A
PUBLIC DOCUMENT COUNT:		4
FILED AS OF DATE:		20180605
DATE AS OF CHANGE:		20180604

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AFRIKA4U
		CENTRAL INDEX KEY:			0001727815
		STANDARD INDUSTRIAL CLASSIFICATION:	WHOLESALE-APPAREL, PIECE GOODS &amp; NOTIONS [5130]
		IRS NUMBER:				000000000
		STATE OF INCORPORATION:			NV
		FISCAL YEAR END:			0831

	FILING VALUES:
		FORM TYPE:		S-1/A
		SEC ACT:		1933 Act
		SEC FILE NUMBER:	333-222638
		FILM NUMBER:		18879441

	BUSINESS ADDRESS:	
		STREET 1:		12 BOMPART STREET, WESTDENE
		CITY:			BLOEMFONTEIN
		STATE:			T3
		ZIP:			9301
		BUSINESS PHONE:		775-321-8275

	MAIL ADDRESS:	
		STREET 1:		12 BOMPART STREET, WESTDENE
		CITY:			BLOEMFONTEIN
		STATE:			T3
		ZIP:			9301
    </acceptance-datetime>
  

In [20]:
p_portion = soup.find_all('p')

In [21]:
p_portion

[<p align="center" style="MARGIN: 0px"> </p>,
 <p style="MARGIN: 0px"> </p>,
 <p align="center" style="MARGIN: 0px"><font size="3"><b>UNITED STATES</b></font></p>,
 <p align="center" style="MARGIN: 0px"><font size="3"><b>SECURITIES AND EXCHANGE COMMISSION</b></font></p>,
 <p align="center" style="MARGIN: 0px"><font size="3"><b>Washington, D.C. 20549</b></font></p>,
 <p align="center" style="MARGIN: 0px"> </p>,
 <p align="center" style="MARGIN: 0px"><font size="5"><b>FORM S-1 A4 </b></font></p>,
 <p align="center" style="MARGIN: 0px">REGISTRATION STATEMENT UNDER THE SECURITIES ACT OF 1933</p>,
 <p style="MARGIN: 0px"> </p>,
 <p align="center" style="MARGIN: 0px"> </p>,
 <p align="center" style="MARGIN: 0px"><b><font size="5">AFRIKA4U</font></b></p>,
 <p align="center" style="MARGIN: 0px">(Exact name of registrant as specified in its charter)</p>,
 <p style="MARGIN: 0px"> </p>,
 <p align="center" style="MARGIN: 0px"><u><b>NEVADA</b></u></p>,
 <p align="center" style="MARGIN: 0px">(State 

In [22]:
soup.find_all('a')

[<a href="#TOC">Table of Contents</a>,
 <a name="PART I—INFORMATION REQUIRED IN PROSPECTUS">PART I—INFORMATION REQUIRED IN PROSPECTUS</a>,
 <a name="ITEM 1. FOREPART OF THE REGISTRATION STATEMENT AND OUTSIDE FRONT COVER PAGE OF PROSPECTUS.">ITEM 1. FOREPART OF THE REGISTRATION STATEMENT AND OUTSIDE FRONT COVER PAGE OF PROSPECTUS.</a>,
 <a href="#TOC">Table of Contents</a>,
 <a name="TOC">TABLE OF CONTENTS</a>,
 <a href="#PART I—INFORMATION REQUIRED IN PROSPECTUS">PART I—INFORMATION REQUIRED IN PROSPECTUS</a>,
 <a href="#ITEM 1. FOREPART OF THE REGISTRATION STATEMENT AND OUTSIDE FRONT COVER PAGE OF PROSPECTUS.">ITEM 1.</a>,
 <a href="#ITEM 1. FOREPART OF THE REGISTRATION STATEMENT AND OUTSIDE FRONT COVER PAGE OF PROSPECTUS.">FOREPART OF THE REGISTRATION STATEMENT AND OUTSIDE FRONT COVER PAGE OF PROSPECTUS.</a>,
 <a href="#ITEM 3. SUMMARY INFORMATION, RISK FACTORS">ITEM 3.</a>,
 <a href="#ITEM 3. SUMMARY INFORMATION, RISK FACTORS">SUMMARY INFORMATION, RISK FACTORS.</a>,
 <a href="#RISK F